In [1]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [2]:
!python setup.py install

running install
running bdist_egg
running egg_info
writing lux.egg-info\PKG-INFO
writing dependency_links to lux.egg-info\dependency_links.txt
writing requirements to lux.egg-info\requires.txt
writing top-level names to lux.egg-info\top_level.txt
reading manifest file 'lux.egg-info\SOURCES.txt'
writing manifest file 'lux.egg-info\SOURCES.txt'
installing library code to build\bdist.win-amd64\egg
running install_lib
running build_py
copying lux\__init__.py -> build\lib\lux
copying sequentially\agent_personal_custom.py -> build\lib\sequentially
copying sequentially\main.py -> build\lib\sequentially
copying sequentially\main_lux.py -> build\lib\sequentially
copying sequentially\__init__.py -> build\lib\sequentially
copying lux\env\agent.py -> build\lib\lux\env
copying lux\env\lux_env.py -> build\lib\lux\env
copying lux\env\__init__.py -> build\lib\lux\env
copying lux\game\actionable.py -> build\lib\lux\game
copying lux\game\actions.py -> build\lib\lux\game
copying lux\game\cell.py -> build

c:\Users\MSII\AppData\Local\Programs\Python\Python311\Lib\site-packages\setuptools\_distutils\cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
c:\Users\MSII\AppData\Local\Programs\Python\Python311\Lib\site-packages\setuptools\_distutils\cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
 

In [3]:
#from stable_baselines3 import PPO  # pip install stable-baselines3
from Algorythm.PPO import PPO
from stable_baselines3.common.callbacks import CheckpointCallback

from importlib import reload
import sequentially.agent_personal_custom
reload(sequentially.agent_personal_custom) # Reload the file from disk incase the above agent-writing cell block was edited
from sequentially.agent_personal_custom import AgentPersonalCustom

from lux.env.agent import Agent
from lux.env.lux_env import LuxEnvironment
from lux.game.constants import LuxMatchConfigs_Default


# Default Lux configs
configs = LuxMatchConfigs_Default

# Create a default opponent agent
opponent = Agent()

# Create a RL agent in training mode
player = AgentPersonalCustom(mode="train")

# Create the Lux environment
env = LuxEnvironment(configs=configs,
                     learning_agent=player,
                     opponent_agent=opponent)

# Define the model, you can pick other RL algos from Stable Baselines3 instead if you like
model = PPO("MlpPolicy",
                env,
                verbose=1,
                tensorboard_log="./lux_tensorboard/",
                learning_rate=0.001,
                gamma=0.999,
                gae_lambda=0.95,
                batch_size=2048 * 8,
                n_steps=2048 * 8,
                device="cuda" if torch.cuda.is_available() else "cpu"
            )

# Define a learning rate schedule
# (number of steps, learning_rate)
schedule = [
    #(2000000, 0.01),
    (2, 0.001),
    (2, 0.0001),
]

c:\Users\MSII\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py:596: UserWarning: WARN: plugin: shimmy.registration:register_gymnasium_envs raised Traceback (most recent call last):
  File "c:\Users\MSII\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py", line 594, in load_plugin_envs
    fn()
  File "c:\Users\MSII\AppData\Local\Programs\Python\Python311\Lib\site-packages\shimmy\registration.py", line 304, in register_gymnasium_envs
    _register_atari_envs()
  File "c:\Users\MSII\AppData\Local\Programs\Python\Python311\Lib\site-packages\shimmy\registration.py", line 205, in _register_atari_envs
    import ale_py
  File "c:\Users\MSII\AppData\Local\Programs\Python\Python311\Lib\site-packages\ale_py\__init__.py", line 66, in <module>
    register_v0_v4_envs()
  File "c:\Users\MSII\AppData\Local\Programs\Python\Python311\Lib\site-packages\ale_py\registration.py", line 176, in register_v0_v4_envs
    _register_r

Running in inference-only mode.
c:\Users\MSII\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [4]:
from stable_baselines3.common.utils import get_schedule_fn
import time

print("Training model...")
start_time = time.time()
run_id = 1

# Save a checkpoint every 1M steps
checkpoint_callback = CheckpointCallback(save_freq=1000000,
                                         save_path='./sequentially/',
                                         name_prefix=f'rl_model_{run_id}')

# Train the policy
for steps, learning_rate in schedule:
    model.lr_schedule = get_schedule_fn(learning_rate)
    model.learn(total_timesteps=steps,
                callback=checkpoint_callback,
                reset_num_timesteps = False)
    
end_time = time.time()
elapsed_time = end_time - start_time

# Save final model
model.save(path=f'sequentially/model.zip')

print("Done training model.")
print(f"Total training time: {elapsed_time} seconds.")

Training model...
Logging to ./lux_tensorboard/PPO_0


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 114      |
|    ep_rew_mean     | 0.309    |
| time/              |          |
|    fps             | 321      |
|    iterations      | 1        |
|    time_elapsed    | 50       |
|    total_timesteps | 16384    |
---------------------------------
Logging to ./lux_tensorboard/PPO_0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 96.3     |
|    ep_rew_mean     | 0.183    |
| time/              |          |
|    fps             | 314      |
|    iterations      | 1        |
|    time_elapsed    | 52       |
|    total_timesteps | 32768    |
---------------------------------
Done training model.
Total training time: 104.14764022827148 seconds.


In [5]:
from kaggle_environments import make
import json
# make match with empty agent
env = make("lux_ai_2021", configuration={"seed": 5621242, "loglevel": 2, "annotations": True}, debug=False)

No pygame installed, ignoring import


In [6]:
# Play the environment where the RL agent plays against itself
steps = env.run(["./sequentially/main.py", "./sequentially/main.py"])

KeyError: 'updates'

In [ ]:
# Render the match
env.render(mode="ipython", width=1200, height=800)